## Исследование продаж по видео играм

<img src='games.jpg' style='width:500px;height:350px'/>

 В комании "Тралала", которая является ведущей в области анализа игр, пришел некоторый заказ на исследование. В котором необходимо найти/рассчитать следующие показатели:

 1. Выполнить поиск игры/игр, которую/ые выпускали на максимальном числе данных нам платформ;
 2. Рассчитать топ-3 и антитоп-3 издателя по количеству дохода используя данные Северной Америки и Японии;
 3. Рассчитать на какую платформу выпустили больше всего игр;
 4. Вычислить сколько денег принесла каждая игра в общемировом масштабе;
 5. Изобразить как росло количество издаваемых игр каждый год.

## Загрузка данных и подготовка к анализу

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
sns.set_style('darkgrid')

In [ ]:
try:
    df_sales = pd.read_csv('game_sales.csv')
except:
    print('Ошибка, проверьте правильность заполненных данных')
else:
    print('Данные успешно загружены')


In [ ]:
def get_info(data):
    display(data.head().style.set_caption('Общая информация по датасету'))
    display(data.describe().T)
    print('----')
    print('Информация о пропусках и типах данных')
    print('----')
    print()
    display(data.info())
    print('----')
    print()

In [ ]:
get_info(df_sales)

-------------------------------------

df_sales — результат запроса. В нём содержится следующая информация:

- Name - Название игры
- Platform - Платформа выпуска игры (т.е. PC, PS4 и т.д.)
- Year - Год выпуска игры
- Genre - Жанр игры
- Publisher - Издатель игры
- NA_Sales - Продажи в Северной Америке (в миллионах)
- EU_Sales - Продажи в Европе (в миллионах)
- JP_Sales - Продажи в Японии (в миллионах)
- Other_Sales - Продажи в остальном мире (в миллионах)

Пропуски наблюдаются в колонках 'Year' - 271 и 'Publisher' - 58, Пропуски изучим и примем решение, что с ними делать. Особой необходимости преобразовывать типы нет, для даты в нашем случае будет подходить и тип float, но для удобства можно преобразовать в int.

Для количества продаж устраивает float64 - вещественные числа. Можно немного уменьшить занимаемую память, понизив тип до float16.

-------------------------------------

### Понижение используемой памяти.

In [ ]:
df_sales = df_sales.astype({'NA_Sales': 'float16', 
                             'EU_Sales': 'float16', 
                             'JP_Sales': 'float16', 
                             'Other_Sales': 'float16'})

### Наличие дубликатов в сете данных.

In [ ]:
df_sales.duplicated().sum()

Найден один дубликат, посмотрим на него.

In [ ]:
df_sales[df_sales.duplicated(keep=False) == True]

Необходимости оставлять дубликат никакой нет, поэтому произведем дроп.

In [ ]:
df_sales = df_sales.drop_duplicates()

In [ ]:
df_sales.duplicated().sum()

Теперь мы можем перейти к пропускам и их обработке

### Столбец Year.

После удаления дубликата на единицу уменьшились пропуски в столбце 'Year'

In [ ]:
print('Количество пропусков в столбце Year', len(df_sales[df_sales['Year'].isna()]))

In [ ]:
df_sales['Year'].unique()

270 пропусков по столбцу год. Проводить иследование и восстанавливать 270 строку слишком громоздко. Заменим на 0, при визуализации исключим из данных строки с этим значением.

In [ ]:
df_sales['Year'] = df_sales['Year'].fillna(0)
print(f'Количество пропусков в столбце Year {len(df_sales[df_sales["Year"].isna()])}')

### Столбец Publisher

In [ ]:
print('Количество пропусков в столбце Publisher', len(df_sales[df_sales['Publisher'].isna()]))

In [ ]:
df_sales[df_sales['Publisher'].isna()].head()

Провели исследование. Обнаружить и привязать данные параметры к Издателю игры не удастся. Заменим название на unknown. Удалять строки не будем. В случае, если будут ошибки, исключим из анализа с помощью запроса.

In [ ]:
df_sales['Publisher'] = df_sales['Publisher'].fillna('unknown')
print('Количество пропусков в столбце Publisher', len(df_sales[df_sales['Publisher'].isna()]))

### Преобразование данных в иные типы

In [ ]:
df_sales = df_sales.astype({'Year': 'int16'}) 

In [ ]:
df_sales.info()

-------------------------------------
Незначительно сэкономили занимаемую память и избавились от nan значений.

-------------------------------------

## 1. Выполнить поиск игры/игр, которую/ые выпускали на максимальном числе данных нам платформ.

In [ ]:
df_sales[df_sales['Name'] == df_sales['Name'].value_counts().idxmax()].sort_values(by='Year')

-------------------------------------
Смотря на вывод, можно обратить внимание на большой промежуток в годах между выпусками. В нашем случае рассматриваются две разные игры, хоть и имеющие одно название. Данное предположение можно проверить с помощью Google. По результату мы можем сказать что данный вывод является неверным и нам не подходит. 

-------------------------------------

<img src='MW_2005.png' style='width:250px;height:350px'/>
<img src='MW_2012.png' style='width:250px;height:350px'/>

In [ ]:
df_sales['Name'].value_counts()[1:]

-------------------------------------
Просмотрев следующее число платформ заметим что у нас идут сразу четыре игры, поэтому мы выведем их все.

-------------------------------------

In [ ]:
df_sales[df_sales['Name'] == df_sales['Name'].value_counts()[1:].idxmax()][['Name', 'Platform']]

In [ ]:
df_sales[df_sales['Name'] == df_sales['Name'].value_counts()[2:].idxmax()][['Name', 'Platform']]

In [ ]:
df_sales[df_sales['Name'] == df_sales['Name'].value_counts()[3:].idxmax()][['Name', 'Platform']]

In [ ]:
df_sales[df_sales['Name'] == df_sales['Name'].value_counts()[4:].idxmax()][['Name', 'Platform']]

-------------------------------------

Лидирующие позиции в самых многоплатформенных играх занимают - 'FIFA 14', 'Ratatouille', 'LEGO Marvel Super Heroes', 'Madden NFL 07'. Так же можно заметить, что названия некоторых платформ есть в одних играх, но нет в других. Поэтому можно сделать вывод, что игры содержат не одно и то же множество платформ.

-------------------------------------

## 2. Рассчитать топ-3 и антитоп-3 издателя по количеству дохода используя данные Северной Америки и Японии.

### Америка

In [ ]:
df_sales.groupby(by='Publisher').sum().nlargest(3, 'NA_Sales')['NA_Sales']

In [ ]:
df_sales.groupby(by='Publisher').sum().nsmallest(3, 'NA_Sales')['NA_Sales']

-------------------------------------

Результаты вывода по Северной Америке показывают, что самые высокие позиции доходах у:
- Nintendo - $817 миллиона

- Electronic Arts - $595 миллиона

- Activision - $429.75 миллиона 

А самые низкие:
- 2D Boy - $0.0 миллиона 

- 49Games - $0.0 миллиона 

- 7G//AMES - $0.0 миллиона 

В результатах мы видим, что у антитопа прослеживаются нулевые доходы. Возможно это связанно с тем, что доходы компании были настолько малы, что данные в итоге не могут поддержаться, либо же эти компании не участвовали на американском рынке.

-------------------------------------

### Япония

In [ ]:
df_sales.groupby(by='Publisher').sum().nlargest(3, 'JP_Sales')['JP_Sales']

In [ ]:
df_sales.groupby(by='Publisher').sum().nsmallest(3, 'JP_Sales')['JP_Sales']

-------------------------------------
Результаты же вывода по Японии показывают что самые высокие позиции в доходах у:
- Nintendo - $455.5 миллиона

- Namco Bandai Games - $127 миллиона

- Konami Digital Entertainment - $91.3 миллиона

А самые низкие:
- 10TACLE Studios - $0.0 миллиона

- 1C Company - $0.0 миллиона

- 20th Century Fox Video Games - $0.0 миллиона

В случае же с самыми малыми доходами мы можем наблюдать ту же картину что и на американском рынке. Либо доходы компании малы, либо они не участвовали в японском рынке

-------------------------------------

## 3. Рассчитать на какую платформу выпустили больше всего игр.

In [ ]:
df_sales.value_counts('Platform').nlargest(1)

In [ ]:
df_sales.value_counts('Platform').nlargest(2)

-------------------------------------
В результате мы можем заметить, что большинство игр вышло на Nintendo DS. Что примечательно, другая платформа 
так же японского производителя - Sony PS2, отстала всего на 2 позиции в нашем DataFrame от лидирующего места. 


-------------------------------------

## 4. Вычислить сколько денег принесла каждая игра в общемировом масштабе.


In [ ]:
df_sales['World_Sales'] = df_sales['NA_Sales'] + df_sales['EU_Sales'] + \
                          df_sales['JP_Sales'] + df_sales['Other_Sales']

In [ ]:
df_sales.head()

-------------------------------------

С помощью сложения каждого столбца по каждой строке мы добились результата характеризующий мировой доход по каждой игре в нашем DataFrame. Теперь, когда нам известен мировой доход, мы сможем вычислять во сколько раз конкретная игра заработала больше/меньше другой или рассчитать процентный доход игры отсносительно издателя который выпустил ее.

-------------------------------------

## 5. Изобразить как росло количество издаваемых игр каждый год.

In [ ]:
plt.figure(figsize=(18, 5), dpi=200)
plt.xticks(rotation=45)
plt.title("Количество издаваемых игр в год")

sns.countplot(data=df_sales[df_sales['Year'] > 0], x='Year')

plt.xlabel("Годы")
plt.ylabel("Количество")

plt.show()

In [ ]:
df_sales.groupby(by='Year').count()['Name'].sort_values(ascending=False).head()

In [ ]:
df_sales.groupby(by='Year').count()['Name'].sort_values(ascending=True).head(5)

------------------------------------
Наиболее результативные года - 2009, 2008

Наименее результативные года - 2020, 2017

Так же на графике мы можем наблюдать как с 1991 года пошел активный рост количества игр, пик которого был в 2009 году и составлял 1431 игры с последующим уменьшением количества до всего одной к 2020 году.


-------------------------------------

## Вывод

- Из csv-файла извлекли необходмые данные - типы данных не требуют преобразования, пропуски были найдены в двух колонка и обработаны.


- Наиболее многоплатформенные игры -  'FIFA 14', 'Ratatouille', 'LEGO Marvel Super Heroes', 'Madden NFL 07'. Эти игры были выпущенны на 9ти различных платформах


- Лидирующие позиции по выручке в:

     - Америке - Nintendo, EA, Activision

     - Японии - Nintendo, Namco Bandai Games , Konami Digital Entertainment
     

- Платформа на которую выпускали чаще всего игры - Nintendo DS. При анализе было обнаруженно, что платформа DS ушла на 2 позиции больше, чем другая платформа от другого японского гиганта Sony - PS2


- Так же было выяснено сколько каждая игра принесла в общемировом масштабе для того, чтобы производить по необходимости более глубокий анализ причастности каждой игры к определенному показателю


- Был сделан показательный график количества выпускаемых игр в каждый год:
    -  2009 и 2008 - самые результативные года с точки зрения количества выпускаемых игр
    -  2020 и 2017 - оказались самыми наименнее результативными годами.